#### SGAN (Semi-supervised generative adversarial network)

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import backend as K

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import (Activation, BatchNormalization, Concatenate,
                                    Dense, Dropout, Flatten, Input, Lambda, Reshape)
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [4]:
# 모델 입력 차원

img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)  # 입력 이미지 차원

z_dim = 100  # 생성자의 입력으로 사용할 잡음 벡터의 크기

num_classes = 10  # 데이터셋에 있는 클래스 개수

In [14]:
# 훈련 데이터셋과 테스트 데이터셋
class Dataset:
    def __init__(self, num_labeled):
        self.num_labeled = num_labeled  # 훈련에 사용할 레이블된 샘플 개수
        # MNIST 데이터셋 적재
        (self.x_train, self.y_train), (self.x_test, self.y_test) = mnist.load_data()

        def preprocess_imgs(x):
            x = (x.astype(np.float32) - 127.5) / 127.5  # (0, 255) 사이 흑백 픽셀 값을 [-1, 1] 사이로 변환
            x = np.expand_dims(x, axis=3)  # 너비 * 높이 * 채널로 이미지 차원을 확장
            return x

        def preprocess_labels(y):
            return y.reshape(-1, 1)
        
        # 훈련 데이터
        self.x_train = preprocess_imgs(self.x_train)
        self.y_train = preprocess_labels(self.y_train)

        # 테스트 데이터
        self.x_test = preprocess_imgs(self.x_test)
        self.y_test = preprocess_labels(self.y_test)

    def batch_labeled(self, batch_size):
        # 레이블된 이미지와 레이블의 랜덤 배치 만들기
        idx = np.random.randint(0, self.num_labeled, batch_size)
        imgs = self.x_train[idx]
        labels = self.y_train[idx]
        return imgs, labels

    def batch_unlabeled(self, batch_size):
        # 레이블이 없는 이미지의 랜덤 배치 만들기
        idx = np.random.randint(self.num_labeled, self.x_train.shape[0],
                                batch_size)
        imgs = self.x_train[idx]
        return imgs

    def training_set(self):
        x_train = self.x_train[range(self.num_labeled)]
        y_train = self.y_train[range(self.num_labeled)]
        return x_train, y_train
    
    def test_set(self):
        return self.x_test, self.y_test


In [15]:
num_labeled = 100  # 사용할 레이블된 샘플 개수(나머지는 레이블 없이 사용)

dataset = Dataset(num_labeled)

In [16]:
# 생성자
def build_generator(z_dim):

    model = Sequential()

    model.add(Dense(256 * 7 * 7, input_dim=z_dim))
    model.add(Reshape((7, 7, 256)))  # 완전 연결 층을 사용해 입력을 7 * 7 * 256 크기 텐서로 변경
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))  # 7 * 7 * 256에서 14 * 14 * 128 텐서로 바꾸는 전치 합성곱 층
    model.add(BatchNormalization())  # 배치 정규화
    
    model.add(LeakyReLU(alpha=0.01))  # LeakyReLU 활성화 함수

    model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))  # 14 * 14 * 128에서 14 * 14 * 64 텐서로 바꾸는 전치 합성곱 층
    model.add(BatchNormalization())  # 배치 정규화

    model.add(LeakyReLU(alpha=0.01))  # LeakyReLU 활성화 함수

    model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))  # 14 * 14 * 64에서 28 * 28 * 1 텐서로 바꾸는 전치 합성곱 층
    model.add(Activation('tanh'))  # tanh 활성화 함수

    return model

In [17]:
# 판별자
def build_discriminator_net(img_shape):

    model = Sequential()

    # 28 * 28 * 1에서 14 * 14 * 32 텐서로 바꾸는 합성곱 층
    model.add(
        Conv2D(32,
               kernel_size=3,
               strides=2,
               input_shape=img_shape,
               padding='same'))

    model.add(LeakyReLU(alpha=0.01))  # LeakyReLU 활성화 함수

    # 14 * 14 * 32에서 7 * 7 * 64 텐서로 바꾸는 합성곱 층
    model.add(
        Conv2D(64,
               kernel_size=3,
               strides=2,
               input_shape=img_shape,
               padding='same'))
    
    model.add(LeakyReLU(alpha=0.01))  # LeakyReLU 활성화 함수

    # 7 * 7 * 64에서 3 * 3 * 128 텐서로 바꾸는 합성곱 층
    model.add(
        Conv2D(128,
               kernel_size=3,
               strides=2,
               input_shape=img_shape,
               padding='same') )

    model.add(LeakyReLU(alpha=0.01))  # LeakyReLU 활성화 함수

    model.add(Dropout(0.5))  # 드롭아웃
    model.add(Flatten())  # 텐서 펼치기
    model.add(Dense(num_classes))  # num_classes개의 뉴런을 가진 완전 연결 층

    return model

In [18]:
# SGAN 판별자: 지도 학습 모델
def build_discriminator_supervised(discriminator_net):

    model = Sequential()

    model.add(discriminator_net)

    model.add(Activation('softmax'))  # 진짜 클래스에 대한 예측 확률을 출력하는 소프트맥스 활성화 함수

    return model

In [19]:
# SGAN 판별자: 비지도 학습 모델
def build_discriminator_unsupervised(discriminator_net):

    model = Sequential()

    model.add(discriminator_net)

    def predict(x):
        prediction = 1.0 - (1.0 /   # 진짜 클래스에 대한 확률 분포를 진짜 대 가짜의 이진 확률 분포로 변환
                            (K.sum(K.exp(x), axis=-1, keepdims=True) + 1.0))
        return prediction

    model.add(Lambda(predict))

    return model

In [20]:
# GAN 모델 구성
def build_gan(generator, discriminator):

    model = Sequential()

    # 생성자와 판별자 모델을 연결하기
    model.add(generator)
    model.add(discriminator)

    return model

# 판별자 기반 모델: 이 층들은 지도 학습 훈련과 비지도 학습 훈련에 공유됩니다
discriminator_net = build_discriminator_net(img_shape)

# 비지도 학습 훈련을 위해 판별자를 만들고 컴파일합니다
discriminator_supervised = build_discriminator_supervised(discriminator_net)
discriminator_supervised.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                            optimizer=Adam(learning_rate=0.0003))
discriminator_unsupervised = build_discriminator_unsupervised(discriminator_net)
discriminator_unsupervised.compile(loss='binary_crossentropy', optimizer=Adam())

# 생성자를 만듭니다
generator = build_generator(z_dim)  # 생성자들 만듭니다

# 생성자 훈련을 위해 판별자의 모델 파라미터를 동결합니다
discriminator_unsupervised.trainable = False

# 생성자를 훈련하기 위해 고정된 판별자를 GAN 모델로 만들고 컴파일합니다.
gan = build_gan(generator, discriminator_unsupervised)
gan.compile(loss='binary_crossentropy', optimizer=Adam())


In [27]:
# 훈련
supervised_losses = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):

    real = np.ones((batch_size, 1))  # 진짜 이미지의 레이블: 모두 1

    fake = np.zeros((batch_size, 1))  # 가짜 이미지의 레이블: 모두 0

    for iteration in range(iterations):
        imgs, labels = dataset.batch_labeled(batch_size)  # 레이블된 샘플을 가져옵니다.

        labels = to_categorical(labels, num_classes=num_classes)  # 레이블을 원-핫 인코딩합니다
        imgs_unlabeled = dataset.batch_unlabeled(batch_size) # 레이블이 없는 샘플을 가져옵니다

        z = np.random.normal(0, 1, (batch_size, z_dim))  # 가짜 이미지의 배치를 생성합니다
        gen_imgs = generator.predict(z, verbose=0)

        # 레이블된 진짜 샘플에서 훈련합니다
        d_loss_supervised, accuracy = discriminator_supervised.train_on_batch(imgs, labels)

        # 레이블이 없는 진짜 샘플에서 훈련합니다
        d_loss_real = discriminator_unsupervised.train_on_batch(imgs_unlabeled, real)

        # 가짜 샘플에서 훈련합니다
        d_loss_fake = discriminator_unsupervised.train_on_batch(gen_imgs, fake)
        d_loss_unsupervised = 0.5 * np.add(d_loss_real, d_loss_fake)

        z = np.random.normal(0, 1, (batch_size, z_dim))  # 가짜 이미지의 배치를 생성합니다
        gen_imgs = generator.predict(z, verbose=0)

        # 생성자를 훈련합니다
        g_loss = gan.train_on_batch(z, np.ones((batch_size, 1)))

        if (iteration + 1) % sample_interval == 0:
            supervised_losses.append(d_loss_supervised)
            iteration_checkpoints.append(iteration + 1)

            print(
                f'{iteration+1} [D 손실: {d_loss_supervised}%, 정확도: [{100 * accuracy}% [D 손실: {d_loss_unsupervised}%] [ G 손실: {g_loss}]'
            )


In [28]:
# 모델 훈련
iterations = 8000
batch_size = 32
sample_interval = 800

train(iterations, batch_size, sample_interval)

KeyboardInterrupt: 

In [29]:
# 정확도 체크
x, y = dataset.test_set()
y = to_categorical(y, num_classes=num_classes)

_, accuracy = discriminator_supervised.evaluate(x, y)  # 테스트 세트에서 분류 정확도 계산
print('테스트 정확도: %.2f%%' %(100 * accuracy))

313/313 [==============================] - 1s 2ms/step - loss: 1.7379 - accuracy: 0.5506
테스트 정확도: 55.06%


In [31]:
mnist_classifier = build_discriminator_supervised(
                            build_discriminator_net(img_shape))
mnist_classifier.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam())